In [119]:
#importing the libraries

import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.datasets import fetch_20newsgroups
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import preprocessing
import pandas as pd
import re
import numpy as np
from nltk.tokenize import  word_tokenize
import nltk
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics, model_selection, feature_extraction, ensemble
import math
from sklearn.pipeline import Pipeline
from scipy import sparse


In [120]:
# pull and clean data

In [121]:
questions = pd.read_json("200k_questions.json")

In [122]:
questions

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",$200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,$200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,$200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",$200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",$200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
216925,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,$2000,Turandot,Double Jeopardy!,4999
216926,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,$2000,a titmouse,Double Jeopardy!,4999
216927,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",$2000,Clive Barker,Double Jeopardy!,4999
216928,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",$2000,Geronimo,Double Jeopardy!,4999


In [123]:
# clean columns
questions.columns = questions.columns.str.strip()

In [124]:
# Check and verify no missing/imcomplete data
questions[(questions.isna().any(axis=1)) & (questions['round'] != 'Final Jeopardy!') & (questions['round'] != 'Tiebreaker')]

,category,air_date,question,value,answer,round,show_number


In [125]:
questions.nunique()

category        27995
air_date         3640
question       216132
value             149
answer          88269
round               4
show_number      3640
dtype: int64

In [126]:
# Clean up Columns
questions['value'] = questions['value'].str.replace("$","")
questions['value'] = questions['value'].str.replace(",","")
questions['value'] = questions['value'].fillna('0')
questions['value'] = questions['value'].astype(int)

In [127]:
questions = questions[questions['question'].str.contains('www')==False].reset_index(drop=True)
questions = questions[questions['category'].str.contains('missing')==False].reset_index(drop=True)
questions

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,"'For the last 8 years of his life, Galileo was...",200,Copernicus,Jeopardy!,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,'No. 2: 1912 Olympian; football star at Carlis...,200,Jim Thorpe,Jeopardy!,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,'The city of Yuma in this state has a record a...,200,Arizona,Jeopardy!,4680
3,THE COMPANY LINE,2004-12-31,"'In 1963, live on ""The Art Linkletter Show"", t...",200,McDonald\'s,Jeopardy!,4680
4,EPITAPHS & TRIBUTES,2004-12-31,"'Signer of the Dec. of Indep., framer of the C...",200,John Adams,Jeopardy!,4680
...,...,...,...,...,...,...,...
206362,RIDDLE ME THIS,2006-05-11,'This Puccini opera turns on the solution to 3...,2000,Turandot,Double Jeopardy!,4999
206363,"""T"" BIRDS",2006-05-11,'In North America this term is properly applie...,2000,a titmouse,Double Jeopardy!,4999
206364,AUTHORS IN THEIR YOUTH,2006-05-11,"'In Penny Lane, where this ""Hellraiser"" grew u...",2000,Clive Barker,Double Jeopardy!,4999
206365,QUOTATIONS,2006-05-11,"'From Ft. Sill, Okla. he made the plea, Arizon...",2000,Geronimo,Double Jeopardy!,4999


In [128]:
import re
import nltk
import string
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/dennissmith/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/dennissmith/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
stopword = nltk.corpus.stopwords.words('english')
wn = nltk.WordNetLemmatizer()
ps = nltk.PorterStemmer()


def clean_message(message):
    # From the last assignment
    message = message.lower()
    message = re.sub(r"http\S+", "", message)
    message = re.sub(r"www.\S+", "", message)
    message_links_removed = "".join([char for char in message if char not in string.punctuation])
    message_cleaned = " ".join([word for word in re.split('\W+', message_links_removed)
        if word not in stopword])
    text = " ".join([wn.lemmatize(word) for word in re.split('\W+', message_cleaned)])
    return text

In [130]:
questions['question'] = questions.apply(lambda row: clean_message(row['question']), axis=1)
questions['answer'] = questions.apply(lambda row: clean_message(row['answer']), axis=1)
questions['category'] = questions.apply(lambda row: clean_message(row['category']), axis=1)

date_threshold = np.datetime64("2000-01-01T00:00:00.000000000")
questions.loc[:, "air_date"] = pd.to_datetime(questions.loc[:, "air_date"].values)
questions.loc[:, "air_date"] = questions.loc[:, "air_date"].apply(lambda x: 1 if x>date_threshold else 0)

### Encode Rounds as explained in EDA notebook
le = preprocessing.LabelEncoder()
questions.loc[:, "round"] = le.fit_transform(questions.loc[:, "round"].values)


In [131]:
questions

,category,air_date,question,value,answer,round,show_number
0,history,1,last 8 year life galileo house arrest espousin...,200,copernicus,2,4680
1,espns top 10 alltime athlete,1,2 1912 olympian football star carlisle indian ...,200,jim thorpe,2,4680
2,everybody talk,1,city yuma state record average 4055 hour sunsh...,200,arizona,2,4680
3,company line,1,1963 live art linkletter show company served b...,200,mcdonalds,2,4680
4,epitaph tribute,1,signer dec indep framer constitution mass seco...,200,john adam,2,4680
...,...,...,...,...,...,...,...
206362,riddle,1,puccini opera turn solution 3 riddle posed her...,2000,turandot,0,4999
206363,bird,1,north america term properly applied 4 specie c...,2000,titmouse,0,4999
206364,author youth,1,penny lane hellraiser grew barber shave anothe...,2000,clive barker,0,4999
206365,quotation,1,ft sill okla made plea arizona land home fathe...,2000,geronimo,0,4999


In [139]:
# Create one row for running regression
questions.loc[:, "question_combined"] = questions.loc[:, "question"] +" "+ questions.loc[:, "answer"] +" "+ questions.loc[:, "category"]
questions

,category,air_date,question,value,answer,round,show_number,question_combined
0,history,1,last 8 year life galileo house arrest espousin...,200,copernicus,2,4680,last 8 year life galileo house arrest espousin...
1,espns top 10 alltime athlete,1,2 1912 olympian football star carlisle indian ...,200,jim thorpe,2,4680,2 1912 olympian football star carlisle indian ...
2,everybody talk,1,city yuma state record average 4055 hour sunsh...,200,arizona,2,4680,city yuma state record average 4055 hour sunsh...
3,company line,1,1963 live art linkletter show company served b...,200,mcdonalds,2,4680,1963 live art linkletter show company served b...
4,epitaph tribute,1,signer dec indep framer constitution mass seco...,200,john adam,2,4680,signer dec indep framer constitution mass seco...
...,...,...,...,...,...,...,...,...
206362,riddle,1,puccini opera turn solution 3 riddle posed her...,2000,turandot,0,4999,puccini opera turn solution 3 riddle posed her...
206363,bird,1,north america term properly applied 4 specie c...,2000,titmouse,0,4999,north america term properly applied 4 specie c...
206364,author youth,1,penny lane hellraiser grew barber shave anothe...,2000,clive barker,0,4999,penny lane hellraiser grew barber shave anothe...
206365,quotation,1,ft sill okla made plea arizona land home fathe...,2000,geronimo,0,4999,ft sill okla made plea arizona land home fathe...


In [156]:
def combine_features(text_vectorizer, df):
    question_combined = text_vectorizer.transform(df.loc[:, "question_combined"].values)

    numeric_feats = np.hstack((
        df.loc[:, "air_date"].values.reshape(-1, 1),
        df.loc[:, "round"].values.reshape(-1, 1)
        )
    )
    return sparse.hstack((question_combined, numeric_feats.astype(object)))


In [158]:
## Create train and test
df_train, df_val = model_selection.train_test_split(questions, train_size=0.7)
    
vectorizer = feature_extraction.text.TfidfVectorizer()
vectorizer.fit(df_train.loc[:, "question_combined"])

# X_train = combine_features(vectorizer, df_train)
# y_train = df_train.loc[:, "value"]
# X_val = combine_features(vectorizer, df_val)
# y_val = df_val.loc[:, "value"]

TfidfVectorizer()

In [ ]:
# Random Forest
rf = ensemble.RandomForestRegressor(n_estimators=100, max_depth=15, n_jobs=10)
rf.fit(X_train, y_train)

### Predictions
y_pred = rf.predict(X_train)
rmse_train = metrics.mean_squared_error(y_train, y_pred, squared=False)
print( f"Achieved RMSE of {rmse_train} on training data")
y_pred = rf.predict(X_val)
rmse_val = metrics.mean_squared_error(y_val, y_pred, squared=False)
print( f"Achieved RMSE of {rmse_val} on validation data")

In [164]:
!pip install transformers

In [167]:
from transformers import pipeline
unmasker = pipeline('fill-mask', model='bert-base-uncased')
unmasker("Artificial Intelligence [MASK] take over the world.")

RuntimeError: Failed to import transformers.models.bert.modeling_tf_bert because of the following error (look up to see its traceback):
cannot import name 'convert_batch_encoding' from 'transformers.tf_utils' (/Users/dennissmith/anaconda3/lib/python3.11/site-packages/transformers/tf_utils.py)